In [43]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [19]:
df = pd.read_csv('../data/zillow_data.csv')
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


In [20]:
df_time_series = pd.DataFrame(index=pd.to_datetime(df.columns[7:]), data=np.ones(len(df.columns)-7))
for i in range(df.shape[0]):
    df_time_series[df['RegionName'][i]] = df.iloc[i,7:]
df_time_series.drop(df_time_series.columns[0],axis=1, inplace=True)
df_time_series

,60657,75070,77494,60614,79936,77084,10467,60640,77449,94109,...,3765,84781,12429,97028,12720,1338,3293,40404,81225,89155
1996-04-01,334200,235700,210400,498100,77300,95000,152900,216500,95400,766000,...,80800,135900,78300,136200,62500,94600,92700,57100,191100,176400
1996-05-01,335400,236900,212200,500900,77300,95200,152700,216700,95600,771100,...,80100,136300,78300,136600,62600,94300,92500,57300,192400,176300
1996-06-01,336500,236700,212200,503100,77300,95400,152600,216900,95800,776500,...,79400,136600,78200,136800,62700,94000,92400,57500,193700,176100
1996-07-01,337600,235400,210700,504600,77300,95700,152400,217000,96100,781900,...,78600,136900,78200,136800,62700,93700,92200,57700,195000,176000
1996-08-01,338500,233300,208300,505500,77400,95900,152300,217100,96400,787300,...,77900,137100,78100,136700,62700,93400,92100,58000,196300,175900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-01,1018700,316600,321200,1299000,120300,162800,414300,777900,172300,3778700,...,123400,257600,171300,341000,122800,216400,213100,130600,694700,348900
2018-01-01,1024400,318100,321200,1302700,120300,162800,413900,778500,173300,3770800,...,124400,258000,172400,342300,123200,213100,213700,131700,706400,350400
2018-02-01,1030700,319600,323000,1306400,120500,162900,411400,780500,174200,3763100,...,125500,260600,173600,345000,123200,209800,218300,132500,705300,353000
2018-03-01,1033800,321100,326900,1308500,121000,163500,413200,782800,175400,3779800,...,126600,264700,175800,348000,120700,209200,222700,133000,681500,356000


In [6]:
df_time_series.isna().sum().sum()

156891

In [26]:
df = df[df['State'] == 'NV']
nv_zipcodes = list(df_nv.RegionName)
nv_zipcodes[0]

89108

In [22]:
df_time_series.fillna(method='bfill', inplace=True)

In [23]:
df_time_series.isna().sum().sum()

0

In [27]:
df_time_series = df_time_series[nv_zipcodes]

In [28]:
size = int(len(df_time_series)*.8)
train = df_time_series.iloc[:size]
test = df_time_series.iloc[size:]

In [31]:
x = 0
train_data = train.iloc[:,x:x+1].values.astype(int)
test_data = test.iloc[:,x:x+1].values.astype(int)

In [32]:
scaler = MinMaxScaler(feature_range=(0,1))
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.fit_transform(test_data)

In [33]:
X_train = []
y_train = []
for i in range(60,len(train_data_scaled)):
    X_train.append(train_data_scaled[i-60:i])
    y_train.append(train_data_scaled[i])

data_total = pd.concat((train.iloc[:,x:x+1], test.iloc[:,x:x+1]),axis=0)
inputs = data_total[len(train)-60:].values
inputs = scaler.transform(inputs)

X_test = []
y_test = []
for i in range(60,len(inputs)):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i])
X_test = np.array(X_test)
y_test = np.array(test_data)

In [34]:
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)

In [44]:
zipcode = 60657

In [59]:
filepath = 'rnn_models/{epoch}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')
callbacks = [checkpoint]

In [66]:
rnn_model = Sequential()
rnn_model.add(LSTM(units= 60, return_sequences = True, input_shape=((60,1))))
rnn_model.add(LSTM(units= 60, return_sequences = False))
rnn_model.add(Dense(units=1))
rnn_model.compile(optimizer='adam', loss='mean_squared_error',metrics=(tf.metrics.MeanAbsolutePercentageError()))
rnn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=.2,callbacks = callbacks)


Epoch 1/100
3/4 [=====================>........] - ETA: 0s - loss: 0.2416 - mean_absolute_percentage_error: 73.8939
Epoch 00001: val_loss did not improve from 0.00159
4/4 [==============================] - 1s 230ms/step - loss: 0.2124 - mean_absolute_percentage_error: 69.3362 - val_loss: 0.0088 - val_mean_absolute_percentage_error: 3462838.0000
Epoch 2/100
3/4 [=====================>........] - ETA: 0s - loss: 0.0525 - mean_absolute_percentage_error: 79.8767
Epoch 00002: val_loss did not improve from 0.00159
4/4 [==============================] - 0s 40ms/step - loss: 0.0583 - mean_absolute_percentage_error: 89.6614 - val_loss: 0.0236 - val_mean_absolute_percentage_error: 6177625.5000
Epoch 3/100
3/4 [=====================>........] - ETA: 0s - loss: 0.0550 - mean_absolute_percentage_error: 100.4747
Epoch 00003: val_loss did not improve from 0.00159
4/4 [==============================] - 0s 39ms/step - loss: 0.0478 - mean_absolute_percentage_error: 87.2539 - val_loss: 0.0092 - val_mean_

In [36]:
y_hat_raw = rnn_model.predict(X_test)
y_hat = scaler.inverse_transform(y_hat_raw)

In [40]:
np.mean(np.absolute((y_hat-y_test)/y_test))

0.020900486578378424

In [53]:
load_file = 'rnn_models/{epoch}.hdf5'
model_resnet28 = load_model(load_file)

In [58]:
model_resnet28.evaluate(X_train,y_train)

5/5 [==============================] - 0s 5ms/step - loss: 7.9643e-04


0.0007964287069626153

In [ ]:
a